In [ ]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
import json
import pandas as pd
import zipfile
import requests
import json


In [ ]:


# GitHub raw file URL
github_file_url = 'https://raw.githubusercontent.com/thousandoaks/Python4DS103/main/data/raw_nyc_phil.json'

# Download the JSON file
response = requests.get(github_file_url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Load JSON content
    raw_nyc_phil = json.loads(response.text)

    # Now you can work with the JSON data
    #print(raw_nyc_phil)
else:
    print(f"Failed to download JSON file. Status code: {response.status_code}")


In [ ]:
type(raw_nyc_phil)

dict

In [ ]:
nycphil = pd.json_normalize(raw_nyc_phil['programs'])
nycphil.head(3)

,season,orchestra,concerts,programID,works,id
0,1842-43,New York Philharmonic,"[{'Date': '1842-12-07T05:00:00Z', 'eventType':...",3853,"[{'workTitle': 'SYMPHONY NO. 5 IN C MINOR, OP....",38e072a7-8fc9-4f9a-8eac-3957905c0002
1,1842-43,New York Philharmonic,"[{'Date': '1843-02-18T05:00:00Z', 'eventType':...",5178,[{'workTitle': 'SYMPHONY NO. 3 IN E FLAT MAJOR...,c7b2b95c-5e0b-431c-a340-5b37fc860b34
2,1842-43,Musicians from the New York Philharmonic,"[{'Date': '1843-04-07T05:00:00Z', 'eventType':...",10785,"[{'workTitle': 'EGMONT, OP.84', 'composerName'...",894e1a52-1ae5-4fa7-aec0-b99997555a37


In [ ]:
pd.options.display.max_colwidth = 500
nycphil.loc[21]

season                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   1847-48
orchestra                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

### We see (at least) two nested columns, concerts and works.
### Let's unpack the works column into a standalone dataframe.


In [ ]:
works_data = pd.json_normalize(data=raw_nyc_phil['programs'], record_path='works',
                            meta=['id', 'orchestra','programID', 'season'])

In [ ]:
works_data

,workTitle,conductorName,ID,soloists,composerName,movement,interval,movement.em,movement._,workTitle.em,workTitle._,id,orchestra,programID,season
0,"SYMPHONY NO. 5 IN C MINOR, OP.67","Hill, Ureli Corelli",52446*,[],"Beethoven, Ludwig van",NaN,NaN,NaN,NaN,NaN,NaN,38e072a7-8fc9-4f9a-8eac-3957905c0002,New York Philharmonic,3853,1842-43
1,OBERON,"Timm, Henry C.",8834*4,"[{'soloistName': 'Otto, Antoinette', 'soloistR...","Weber, Carl Maria Von","""Ozean, du Ungeheuer"" (Ocean, thou mighty mons...",NaN,NaN,NaN,NaN,NaN,38e072a7-8fc9-4f9a-8eac-3957905c0002,New York Philharmonic,3853,1842-43
2,"QUINTET, PIANO, D MINOR, OP. 74",NaN,3642*,"[{'soloistName': 'Scharfenberg, William', 'sol...","Hummel, Johann",NaN,NaN,NaN,NaN,NaN,NaN,38e072a7-8fc9-4f9a-8eac-3957905c0002,New York Philharmonic,3853,1842-43
3,NaN,NaN,0*,[],NaN,NaN,Intermission,NaN,NaN,NaN,NaN,38e072a7-8fc9-4f9a-8eac-3957905c0002,New York Philharmonic,3853,1842-43
4,OBERON,"Etienne, Denis G.",8834*3,[],"Weber, Carl Maria Von",Overture,NaN,NaN,NaN,NaN,NaN,38e072a7-8fc9-4f9a-8eac-3957905c0002,New York Philharmonic,3853,1842-43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82788,MEMORIES OF MARNYI STONE,"Gersen, Joshua",12636*,[],"Feng, Shuya",NaN,NaN,NaN,NaN,NaN,NaN,68625626-8085-44e4-bdb3-d8e6ac51c95b,New York Philharmonic,14136,2016-17
82789,"SYMPHONY NO. 9, E MINOR, OP.95 (FROM THE NEW W...","Gersen, Joshua",2877*4,[],"Dvorak, Antonín",Allegro con fuoco,NaN,NaN,NaN,NaN,NaN,68625626-8085-44e4-bdb3-d8e6ac51c95b,New York Philharmonic,14136,2016-17
82790,QUIET CITY,"Gilbert, Alan",52332*,"[{'soloistName': 'Martin, Christopher', 'soloi...","Copland, Aaron",NaN,NaN,NaN,NaN,NaN,NaN,5c22d35f-a282-4505-bd31-df619c74f52b,New York Philharmonic,14147,2016-17
82791,"JUNGLE, THE (SYMPHONY NO. 4)","Gilbert, Alan",12071*,[{'soloistName': 'Jazz at Lincoln Center Orche...,"Marsalis, Wynton",NaN,NaN,NaN,NaN,NaN,NaN,5c22d35f-a282-4505-bd31-df619c74f52b,New York Philharmonic,14147,2016-17


#### Great! We: (1)passed the json object data path d[programs],(2) passed the record path within the object we wanted to parse works, (3)passed the parent metadata we wanted to append


#### So what if you run into a nested array inside your nested array? If you go back and look at the flattened works_data, you can see a second nested column, soloists. Luckily, json_normalize docs show that you can pass in a list of columns, rather than a single column, to the record path to directly unflatten deeply nested json.

#### Let's flatten the 'soloists' data here by passing a list. Since soloists is nested in works, we can pass that as:

In [ ]:
soloist_data = pd.json_normalize(data=raw_nyc_phil['programs'], record_path=['works', 'soloists'],
                              meta=['id'])
soloist_data

,soloistName,soloistRoles,soloistInstrument,id
0,"Otto, Antoinette",S,Soprano,38e072a7-8fc9-4f9a-8eac-3957905c0002
1,"Scharfenberg, William",A,Piano,38e072a7-8fc9-4f9a-8eac-3957905c0002
2,"Hill, Ureli Corelli",A,Violin,38e072a7-8fc9-4f9a-8eac-3957905c0002
3,"Derwort, G. H.",A,Viola,38e072a7-8fc9-4f9a-8eac-3957905c0002
4,"Boucher, Alfred",A,Cello,38e072a7-8fc9-4f9a-8eac-3957905c0002
...,...,...,...,...
56189,"Xu, Weiqin",S,Baritone,68625626-8085-44e4-bdb3-d8e6ac51c95b
56190,"Martin, Christopher",S,Trumpet,5c22d35f-a282-4505-bd31-df619c74f52b
56191,"Shryock, Grace",S,English Horn,5c22d35f-a282-4505-bd31-df619c74f52b
56192,Jazz at Lincoln Center Orchestra with Wynton M...,S,Jazz Ensemble,5c22d35f-a282-4505-bd31-df619c74f52b
